In [ ]:
Testing Pre-made Quantopian

In [ ]:
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline
from quantopian.pipeline.factors import Latest
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.data import morningstar
from quantopian.pipeline.factors import CustomFactor, SimpleMovingAverage, AverageDollarVolume, Returns, RSI
from quantopian.pipeline.classifiers.morningstar import Sector
from quantopian.pipeline.filters import QTradableStocksUS
#from quantopian.pipeline.data.quandl import fred_usdontd156n as libor
from quantopian.pipeline.data.zacks import EarningsSurprises

import talib
import pandas as pd
import numpy as np

import alphalens as al
import pyfolio as pf
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import linear_model, decomposition, ensemble, preprocessing, isotonic, metrics

bs = morningstar.balance_sheet
cfs = morningstar.cash_flow_statement
is_ = morningstar.income_statement
or_ = morningstar.operation_ratios
er = morningstar.earnings_report
v = morningstar.valuation
vr = morningstar.valuation_ratios

In [ ]:
def make_factors():
    def Price_To_Book():
            """
            Price to Book Value:
            Closing price divided by book value.
            https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
            Notes:
            Low P/B Ratio suggests that equity is cheap
            Differs substantially between sectors
            """
            return USEquityPricing.close.latest / \
                vr.book_value_per_share.latest

    def Price_To_Earnings():
            """
            Price to Earnings Ratio:
            Closing price divided by earnings per share.
            https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
            Notes:
            Low P/E Ratio suggests that an equity cheap
            Differs substantially between sectors
            """
            return vr.pe_ratio.latest

    def EV_To_EBITDA():
            """
            Enterprise Value to Earnings Before Interest, Taxes,
            Deprecation and Amortization (EBITDA):
            Enterprise Value divided by EBITDA.
            https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
            Notes:
            Low EV/EBITDA suggests that equity is cheap
            Differs substantially between sectors / companies
            """
            return v.enterprise_value.latest / \
                (is_.ebitda.latest * 4.)

    """RISK/SIZE"""

    class Downside_Risk(CustomFactor):
        """
        Downside Risk:
        Standard Deviation of 12-month monthly losses
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests high risk of losses
        """
        inputs = [Returns(window_length=2)]
        window_length = 252

        def compute(self, today, assets, out, rets):
            down_rets = np.where(rets < 0, rets, np.nan)
            out[:] = np.nanstd(down_rets, axis=0)

    def SPY_Beta():
        """
        Index Beta:
        Slope coefficient of 1-year regression of price returns
        against index returns
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests high market risk
        Slope calculated using regression MLE
        """
        return RollingLinearRegressionOfReturns(
            target=symbols('SPY'),
            # above for research
            # target=sid(8554) for backtester
            returns_length=2,
            regression_length=252
        ).beta

    """GROWTH"""

    def Sales_Growth_3M():
        """
        3-month Sales Growth:
        Increase in total sales over 3 months
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value represents large growth (short term)
        """
        return Returns(inputs=[is_.total_revenue], window_length=63)

    def Sales_Growth_12M():
        """
        12-month Sales Growth:
        Increase in total sales over 12 months
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value represents large growth (long term)
        """
        return Returns(inputs=[is_.total_revenue], window_length=252)

    def EPS_Growth_12M():
        """
        12-month Earnings Per Share Growth:
        Increase in EPS over 12 months
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value represents large growth (long term)
        """
        return Returns(inputs=[er.basic_eps], window_length=252)

    """"QUALITY"""

    class Asset_Turnover(CustomFactor):
        """
        Asset Turnover:
        Sales divided by average of year beginning and year end assets
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value represents good financial health
        Varies substantially between sectors
        """
        inputs = [is_.total_revenue, bs.total_assets]
        window_length = 252

        def compute(self, today, assets, out, sales, tot_assets):
            out[:] = (sales[-1] * 4.) / \
                ((tot_assets[-1] + tot_assets[0]) / 2.)

    def Asset_Growth_3M():
        """
        3-month Asset Growth:
        Increase in total assets over 3 months
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value represents good financial health as quantity of
        assets is increasing
        """
        return Returns(inputs=[bs.total_assets], window_length=63)

    def Current_Ratio():
        """
        Current Ratio:
        Total current assets divided by total current liabilities
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value represents good financial health as assets
        are greater than liabilities (>1)
        Morningstar built-in fundamental ratio more accurate
        than calculated ratio
        """
        return or_.current_ratio.latest

    def Asset_To_Equity_Ratio():
        """
        Asset / Equity Ratio
        Total current assets divided by common equity
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests that company has taken on substantial debt
        Vaires substantially with industry
        """
        return bs.total_assets.latest / bs.common_stock_equity.latest

    def Interest_Coverage():
        """
        Interest Coverage:
        EBIT divided by interest expense
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests that company has taken on substantial debt
        Varies substantially with industry
        """
        return is_.ebit.latest / is_.interest_expense.latest

    def Debt_To_Asset_Ratio():
        """
        Debt / Asset Ratio:
        Total Debts divided by Total Assets
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests that company has taken on substantial debt
        Low value suggests good financial health as assets greater than debt
        Long Term Debt
        """
        return bs.total_debt.latest / bs.total_assets.latest

    def Debt_To_Equity_Ratio():
        """
        Debt / Equity Ratio:
        Total Debts divided by Common Stock Equity
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests that company is taking on debts to leverage
        Low value suggests good financial health as little-to-no leveraging
        Long Term Debt
        """
        return bs.total_debt.latest / bs.common_stock_equity.latest

    class Net_Debt_Growth_3M(CustomFactor):
        """
        3-Month Net Debt Growth:
        Increase in net debt (total debt - cash) over 3 month window
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests that a company is acquiring more debt
        Low value suggests good financial health as debt is decreasing
        Long Term Debt
        """
        inputs = [bs.total_debt, bs.cash_and_cash_equivalents]
        window_length = 92

        def compute(self, today, assets, out, debt, cash):
            out[:] = ((debt[-1] - cash[-1]) - (debt[0] - cash[0])) / \
                (debt[0] - cash[0])

    def Working_Capital_To_Assets():
        """
        Working Capital / Assets:
        Current Assets less Current liabilities (Working Capital)
        all divided by Assets
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests that company is currently using more money
        than they are saving (holding as assets)
        Low value suggests good financial health as holding more in assets
        than risking through use working_capital more accurate 
        than total_assets and total_liabilities
        """
        return bs.working_capital.latest / bs.total_assets.latest

    def Working_Capital_To_Sales():
        """
        Working Capital / Sales:
        Current Assets less Current liabilities (Working Capital)
        all divided by Sales
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests that company is currently using more money
        than taking in through sales than they are saving (holding as assets)
        Low value suggests good financial health as sales stronger
        than money risked through use
        working_capital more accurate than total_assets and total_liabilities
        """
        return bs.working_capital.latest / is_.total_revenue.latest


    def Earnings_Quality():
        return morningstar.cash_flow_statement.operating_cash_flow.latest / \
               EarningsSurprises.eps_act.latest
    """PROFITABILITY"""

    def Gross_Income_Margin():
        """
        Gross Income Margin:
        Gross Profit divided by Net Sales
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests that the company is generating large profits
        """
        return is_.gross_profit.latest / is_.total_revenue.latest

    def Net_Income_Margin():
        """
        Gross Income Margin:
        Gross Profit divided by Net Sales
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests that the company is generating large profits
        Builtin used as cleans inf values
        """
        return or_.net_margin.latest

    def Return_On_Total_Equity():
        """
        Return on Total Equity:
        Net income divided by average of total shareholder equity
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests that the company is generating large profits
        Builtin used as cleans inf values
        """
        return or_.roe.latest

    def Return_On_Total_Assets():
        """
        Return on Total Assets:
        Net income divided by average total assets
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests that the company is generating large profits
        Builtin used as cleans inf values
        """
        return or_.roa.latest
    
    all_factors = {
        'Asset Growth 3M': Asset_Growth_3M,
        'Asset to Equity Ratio': Asset_To_Equity_Ratio,
        'Asset Turnover': Asset_Turnover,
        'Capex to Assets': Capex_To_Assets,
        'Capex to Cashflows': Capex_To_Cashflows,
        'Capex to Sales': Capex_To_Sales,
        'Cashflows to Assets': Cashflows_To_Assets,
        'Current Ratio': Current_Ratio,
        'Debt to Asset Ratio': Debt_To_Asset_Ratio,
        'Dividend Growth': Dividend_Growth,
        'Dividend Payout Ratio': Dividend_Payout_Ratio,
        'Dividend Yield': Dividend_Yield,
        'EBITDA Yield': EBITDA_Yield,
        'EBIT to Assets': EBIT_To_Assets,
        'Earnings Quality': Earnings_Quality,
        'EV to Cashflows': EV_To_Cashflows,
        'EV to EBITDA': EV_To_EBITDA,
        'Gross Income Margin': Gross_Income_Margin,
        'Interest Coverage': Interest_Coverage,
        'MACD Signal Line': MACD_Signal_10d,
        'Market Cap': Market_Cap,
        'Mean Reversion 1M': Mean_Reversion_1M,
        'Moneyflow Volume 5D': Moneyflow_Volume_5d,
        'Net Debt Growth 3M': Net_Debt_Growth_3M,
        'Net Income Margin': Net_Income_Margin,
        'Operating Cashflows to Assets': Operating_Cashflows_To_Assets,
        'Price Momentum 12M': Price_Momentum_12M,
        'Price Momentum 1M': Price_Momentum_1M,
        'Price Momentum 3M': Price_Momentum_3M,
        'Price Momentum 6M': Price_Momentum_6M,
        'Price Oscillator': Price_Oscillator,
        'Price to Book': Price_To_Book,
        'Price to Diluted Earnings': Price_To_Diluted_Earnings,
        'Price to Earnings': Price_To_Earnings,
        'Price to Forward Earnings': Price_To_Forward_Earnings,
        'Price to Free Cashflows': Price_To_Free_Cashflows,
        'Price to Operating Cashflows': Price_To_Operating_Cashflows,
        'Price to Sales': Price_To_Sales,
        'Retained Earnings to Assets': Retained_Earnings_To_Assets,
        'Return on Total Assets': Return_On_Total_Assets,
        'Return on Total Equity': Return_On_Total_Equity,
        'Return on Invest Capital': Return_On_Total_Invest_Capital,
        '39 Week Returns': Returns_39W,
        'Stochastic Oscillator': Stochastic_Oscillator,
        'Trendline': Trendline,
        'Vol 3M': Vol_3M,
        'Working Capital to Assets': Working_Capital_To_Assets,
        'Working Capital to Sales': Working_Capital_To_Sales,
    }
    
    return all_factors